In [1]:
'''
import libraries
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
# nltk.download('all')
# Add the NLTK data directory to the paths
# nltk.data.path.append('/Library/Frameworks/Python.framework/Versions/3.12/nltk_data')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# !python3 -m pip install --upgrade nltk


In [3]:
# !python3 -m pip install nltk
# !python3 -m nltk.downloader punkt
# !python3 -m nltk.downloader stopwords

# !python3 -m pip install spacy

In [4]:
'''
Load data
'''
data_dir = 'data/input/collection_with_abstracts.csv'
df = pd.read_csv(data_dir)
df.head()

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI,Abstract
0,39435445,Editorial: The operationalization of cognitive...,"Winter M, Probst T, Tallon M, Schobel J, Pryss R.",Front Neurosci. 2024 Oct 7;18:1501636. doi: 10...,Winter M,Front Neurosci,2024,2024/10/22,PMC11491427,NaN,10.3389/fnins.2024.1501636,NaN
1,39398866,Characterization of arteriosclerosis based on ...,"Zhou J, Li X, Demeke D, Dinh TA, Yang Y, Janow...",J Med Imaging (Bellingham). 2024 Sep;11(5):057...,Zhou J,J Med Imaging (Bellingham),2024,2024/10/14,PMC11466048,NaN,10.1117/1.JMI.11.5.057501,PURPOSE: Our purpose is to develop a computer ...
2,39390053,Multi-scale input layers and dense decoder agg...,"Lan X, Jin W.",Sci Rep. 2024 Oct 10;14(1):23729. doi: 10.1038...,Lan X,Sci Rep,2024,2024/10/10,PMC11467340,NaN,10.1038/s41598-024-74701-0,Accurate segmentation of COVID-19 lesions from...
3,39367648,An initial game-theoretic assessment of enhanc...,"Fatemi MY, Lu Y, Diallo AB, Srinivasan G, Azhe...",Brief Bioinform. 2024 Sep 23;25(6):bbae476. do...,Fatemi MY,Brief Bioinform,2024,2024/10/05,PMC11452536,NaN,10.1093/bib/bbae476,The application of deep learning to spatial tr...
4,39363262,Truncated M13 phage for smart detection of E. ...,"Yuan J, Zhu H, Li S, Thierry B, Yang CT, Zhang...",J Nanobiotechnology. 2024 Oct 3;22(1):599. doi...,Yuan J,J Nanobiotechnology,2024,2024/10/04,PMC11451008,NaN,10.1186/s12951-024-02881-y,BACKGROUND: The urgent need for affordable and...


In [5]:
df

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI,Abstract
0,39435445,Editorial: The operationalization of cognitive...,"Winter M, Probst T, Tallon M, Schobel J, Pryss R.",Front Neurosci. 2024 Oct 7;18:1501636. doi: 10...,Winter M,Front Neurosci,2024,2024/10/22,PMC11491427,NaN,10.3389/fnins.2024.1501636,NaN
1,39398866,Characterization of arteriosclerosis based on ...,"Zhou J, Li X, Demeke D, Dinh TA, Yang Y, Janow...",J Med Imaging (Bellingham). 2024 Sep;11(5):057...,Zhou J,J Med Imaging (Bellingham),2024,2024/10/14,PMC11466048,NaN,10.1117/1.JMI.11.5.057501,PURPOSE: Our purpose is to develop a computer ...
2,39390053,Multi-scale input layers and dense decoder agg...,"Lan X, Jin W.",Sci Rep. 2024 Oct 10;14(1):23729. doi: 10.1038...,Lan X,Sci Rep,2024,2024/10/10,PMC11467340,NaN,10.1038/s41598-024-74701-0,Accurate segmentation of COVID-19 lesions from...
3,39367648,An initial game-theoretic assessment of enhanc...,"Fatemi MY, Lu Y, Diallo AB, Srinivasan G, Azhe...",Brief Bioinform. 2024 Sep 23;25(6):bbae476. do...,Fatemi MY,Brief Bioinform,2024,2024/10/05,PMC11452536,NaN,10.1093/bib/bbae476,The application of deep learning to spatial tr...
4,39363262,Truncated M13 phage for smart detection of E. ...,"Yuan J, Zhu H, Li S, Thierry B, Yang CT, Zhang...",J Nanobiotechnology. 2024 Oct 3;22(1):599. doi...,Yuan J,J Nanobiotechnology,2024,2024/10/04,PMC11451008,NaN,10.1186/s12951-024-02881-y,BACKGROUND: The urgent need for affordable and...
...,...,...,...,...,...,...,...,...,...,...,...,...
11445,10607521,The characteristics of epidemics and invasions...,"Cruickshank I, Gurney WS, Veitch AR.",Theor Popul Biol. 1999 Dec;56(3):279-92. doi: ...,Cruickshank I,Theor Popul Biol,1999,1999/12/23,NaN,NaN,10.1006/tpbi.1999.1432,In this paper we report the development of a h...
11446,10072741,Effects of sales promotion on smoking among U....,Redmond WH.,Prev Med. 1999 Mar;28(3):243-50. doi: 10.1006/...,Redmond WH,Prev Med,1999,1999/03/12,NaN,NaN,10.1006/pmed.1998.0410,OBJECTIVE: The purpose of this study was to ex...
11447,9200018,Hypertension in an inner-city minority population,Wieck KL.,J Cardiovasc Nurs. 1997 Jul;11(4):41-9. doi: 1...,Wieck KL,J Cardiovasc Nurs,1997,1997/07/01,NaN,NaN,10.1097/00005082-199707000-00005,This study describes an inner-city elderly min...
11448,8039948,Aerosol transmission of a viable virus affecti...,"Grant RH, Scheidt AB, Rueff LR.",Int J Biometeorol. 1994 May;38(1):33-9. doi: 1...,Grant RH,Int J Biometeorol,1994,1994/05/01,NaN,NaN,10.1007/BF01241802,A Gaussian diffusion model was applied to an e...


In [6]:
'''
Check for missing values
'''
df['Abstract'].isna().sum()

213

In [7]:
'''
check for duplicates
'''
df['DOI'].nunique() 

10969

In [8]:
'''
missing values in Abstract column
'''

nrow = df.shape[0]
nnullAb = df.Abstract.isna().sum()
(nnullAb/nrow ) * 100

1.8602620087336246

approx 1.86% of the data is missing in the Abstract column

In [9]:
# Drop rows where 'Abstract' column has missing values
print(f'df before removal::{df.shape}')
df_cleaned = df.dropna(subset=['Abstract'])
print(f'df after removal::{df_cleaned.shape}')
print(f'# rows removed = {df.shape[0] - df_cleaned.shape[0]}')

df before removal::(11450, 12)
df after removal::(11237, 12)
# rows removed = 213


In [10]:
from pprint import pprint

In [11]:

# df[['DOI', 'Abstract']].isnull().sum()
# df_abs_nnan = df[df['Abstract'].isnull()]
# df_abs_nnan[['DOI', 'Abstract']].isna().sum()
# !python3 -m spacy download en_core_web_sm

# !python3 -m pip install --upgrade pip


In [12]:
df_cleaned['titleAbstract'] = df_cleaned['Title'] + ' ' + df_cleaned['Abstract']
df_cleaned.head(2)



/var/folders/mw/z0lcvm_j57l5gjx8mshnsnkm0000gn/T/ipykernel_22220/530045461.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['titleAbstract'] = df_cleaned['Title'] + ' ' + df_cleaned['Abstract']


,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI,Abstract,titleAbstract
1,39398866,Characterization of arteriosclerosis based on ...,"Zhou J, Li X, Demeke D, Dinh TA, Yang Y, Janow...",J Med Imaging (Bellingham). 2024 Sep;11(5):057...,Zhou J,J Med Imaging (Bellingham),2024,2024/10/14,PMC11466048,NaN,10.1117/1.JMI.11.5.057501,PURPOSE: Our purpose is to develop a computer ...,Characterization of arteriosclerosis based on ...
2,39390053,Multi-scale input layers and dense decoder agg...,"Lan X, Jin W.",Sci Rep. 2024 Oct 10;14(1):23729. doi: 10.1038...,Lan X,Sci Rep,2024,2024/10/10,PMC11467340,NaN,10.1038/s41598-024-74701-0,Accurate segmentation of COVID-19 lesions from...,Multi-scale input layers and dense decoder agg...


In [13]:
import re
# import spacy

In [14]:
# !python3 -m pip install nltk

In [15]:
# from nltk.tokenize import word_tokenize

# Example text
text = "This is a sentence test."
tokens = word_tokenize(text)
print(tokens)


['This', 'is', 'a', 'sentence', 'test', '.']


In [16]:
df_cleaned_lem = df_cleaned.copy()

en_stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    text = re.sub(r'[^A-Za-z1-9 ]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)

    clean_text = []
    for token in tokens:
        if token not in en_stopwords:
            clean_text.append(lemmatizer.lemmatize(token))
    return ' '.join(clean_text)

In [17]:
df_cleaned['titleAbstractNew'] = df_cleaned['titleAbstract'].apply(lemmatize_text)

/var/folders/mw/z0lcvm_j57l5gjx8mshnsnkm0000gn/T/ipykernel_22220/3072049966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['titleAbstractNew'] = df_cleaned['titleAbstract'].apply(lemmatize_text)


In [18]:
df_cleaned.head(2)

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI,Abstract,titleAbstract,titleAbstractNew
1,39398866,Characterization of arteriosclerosis based on ...,"Zhou J, Li X, Demeke D, Dinh TA, Yang Y, Janow...",J Med Imaging (Bellingham). 2024 Sep;11(5):057...,Zhou J,J Med Imaging (Bellingham),2024,2024/10/14,PMC11466048,NaN,10.1117/1.JMI.11.5.057501,PURPOSE: Our purpose is to develop a computer ...,Characterization of arteriosclerosis based on ...,characterization arteriosclerosis based comput...
2,39390053,Multi-scale input layers and dense decoder agg...,"Lan X, Jin W.",Sci Rep. 2024 Oct 10;14(1):23729. doi: 10.1038...,Lan X,Sci Rep,2024,2024/10/10,PMC11467340,NaN,10.1038/s41598-024-74701-0,Accurate segmentation of COVID-19 lesions from...,Multi-scale input layers and dense decoder agg...,multiscale input layer dense decoder aggregati...


In [19]:

# Define keywords for filtering

keywords = [
    # Deep Learning Specific
    'deep learning', 'deep neural networks', 'neural network', 'artificial neural network',  
    'feedforward neural network', 'neural net algorithm', 
    'multilayer perceptron', 'convolutional neural network', 'cnn', 
    'recurrent neural network', 'rnn', 'long short-term memory network', 
    'lstm', 'transformer', 'transformer models', 'self-attention models', 
    'attention-based neural networks',
    
    # Computer Vision
    'computer vision', 'vision model', 'image processing', 
    'vision algorithms', 'object recognition', 'scene understanding',

    # Natural Language Processing
    'natural language processing', 'text mining', 'NLP', 
    'computational linguistics', 'language processing', 
    'text analytics', 'textual data analysis', 'text data analysis', 
    'text analysis', 'speech and language technology', 
    'language modeling', 'computational semantics',

    # Generative AI
    'generative artificial intelligence', 'generative AI', 
    'generative deep learning', 'generative models',

    # Large Language Models
    'large language model', 'transformer-based model', 
    'pretrained language model', 'generative language model', 
    'foundation model', 'state-of-the-art language model',

    # Multimodal Models
    'multimodal model', 'multimodal neural network', 
    'vision transformer', 'diffusion model', 
    'generative diffusion model', 'diffusion-based generative model', 
    'continuous diffusion model'
]



In [24]:
# Reset the index of the DataFrame
df_cleaned.reset_index(drop=True, inplace=True)

In [25]:
df_cleaned.head(2)

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI,Abstract,titleAbstract,titleAbstractNew
0,39398866,Characterization of arteriosclerosis based on ...,"Zhou J, Li X, Demeke D, Dinh TA, Yang Y, Janow...",J Med Imaging (Bellingham). 2024 Sep;11(5):057...,Zhou J,J Med Imaging (Bellingham),2024,2024/10/14,PMC11466048,NaN,10.1117/1.JMI.11.5.057501,PURPOSE: Our purpose is to develop a computer ...,Characterization of arteriosclerosis based on ...,characterization arteriosclerosis based comput...
1,39390053,Multi-scale input layers and dense decoder agg...,"Lan X, Jin W.",Sci Rep. 2024 Oct 10;14(1):23729. doi: 10.1038...,Lan X,Sci Rep,2024,2024/10/10,PMC11467340,NaN,10.1038/s41598-024-74701-0,Accurate segmentation of COVID-19 lesions from...,Multi-scale input layers and dense decoder agg...,multiscale input layer dense decoder aggregati...


In [29]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load the pre-trained SciBERT model
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

def get_embedding(text):
    # Tokenize and get embeddings
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Test embedding function on the title and abstract of the first paper
example_title_embedding = get_embedding(df_cleaned['Title'][0])
example_abstract_embedding = get_embedding(df_cleaned['Abstract'][0])
example_titleabstract_embedding = get_embedding(df_cleaned['titleAbstract'][0])
example_titleabstractnew_embedding = get_embedding(df_cleaned['titleAbstractNew'][0])

print("Title Embedding Shape:", example_title_embedding.shape)
print("Abstract Embedding Shape:", example_abstract_embedding.shape)


Title Embedding Shape: (1, 768)
Abstract Embedding Shape: (1, 768)


Expected Output: You should see the shape of the embeddings printed out, confirming that you have successfully obtained embeddings for the title and abstract. Typically, the shape will be (1, 768) for SciBERT, indicating 768-dimensional embeddings.

# Step 4: Filtering with Semantic Similarity

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

# Create a query vector from the keywords
query_vector = get_embedding(" ".join(keywords))

In [ ]:
# Calculate embeddings for each paper's title and abstract
# df_cleaned['Title_Embedding'] = df_cleaned['Title'].apply(get_embedding)
# df_cleaned['Abstract_Embedding'] = df_cleaned['Abstract'].apply(get_embedding)
# df_cleaned['titleAbstract_Embedding'] = df_cleaned['titleAbstract'].apply(get_embedding)
df_cleaned['titleAbstractNew_Embedding'] = df_cleaned['titleAbstractNew'].apply(get_embedding)
print(f'running...')
# Calculate cosine similarity
# df_cleaned['Title_Similarity'] = df_cleaned['Title_Embedding'].apply(lambda x: cosine_similarity(x, query_vector).flatten())
# df_cleaned['Abstract_Similarity'] = df_cleaned['Abstract_Embedding'].apply(lambda x: cosine_similarity(x, query_vector).flatten())
# df_cleaned['titleAbstract_Similarity'] = df_cleaned['titleAbstract_Embedding'].apply(lambda x: cosine_similarity(x, query_vector).flatten())
df_cleaned['titleAbstractNew_Similarity'] = df_cleaned['titleAbstractNew_Embedding'].apply(lambda x: cosine_similarity(x, query_vector).flatten())

# Display similarity scores
# df_cleaned[['Title', 'Title_Similarity', 'Abstract_Similarity', 'titleAbstract_Similarity', 'titleAbstractNew_Similarity']].head(2)

In [34]:
df_cleaned.columns

Index(['PMID', 'Title', 'Authors', 'Citation', 'First Author', 'Journal/Book',
       'Publication Year', 'Create Date', 'PMCID', 'NIHMS ID', 'DOI',
       'Abstract', 'titleAbstract', 'titleAbstractNew'],
      dtype='object')

In [29]:
# !python3 -m pip install transformers
# !python3 -m pip install torch